## Spotify Transformation Notebook

####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 5.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.8 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 5.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 2880
Session ID: c8ae0d0e-eba8-4ee9-b71b-421a2a134ced
Applying the following default arguments:
--glue_kernel_version 1.0.8
--enable-glue-datacatalog true
Waiting for session c8ae0d0e-eba8-4ee9-b71b-421a2a134ced to get into ready status...
Session c8ae0d0e-eba8-4ee9-b71b-421a2a134ced ha

### Import Libraries

In [9]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql.functions import explode, col, to_date
from datetime import datetime
from awsglue.dynamicframe import DynamicFrame

### Source S3 path

In [ ]:
s3_path = "s3://spotify-bucket-jay/staging_data/raw/"

# Load raw data from S3

source_dymf = glueContext.create_dynamic_frame_from_options(
    connection_type="s3",
    connection_options={"paths":[s3_path]},
    format="json"
)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.8 
Trying to create a Glue session for the kernel.
Session Type: glueetl
Session ID: ba2e12e0-7b54-4999-955a-99d07b88adcf
Applying the following default arguments:
--glue_kernel_version 1.0.8
--enable-glue-datacatalog true
Waiting for session ba2e12e0-7b54-4999-955a-99d07b88adcf to get into ready status...


In [5]:
spotify_df = source_dymf.toDF()

### Transform Data

In [37]:

# Processing Functions

def process_albums(df):
    df_albums = df.withColumn("items", explode("items")).select(
        col("items.track.album.id").alias("album_id"),
        col("items.track.album.name").alias("album_name"),
        col("items.track.album.release_date").alias("release_date"),
        col("items.track.album.total_tracks").alias("total_tracks"),
        col("items.track.album.external_urls.spotify").alias("url")
    ).drop_duplicates(["album_id"])
    
    return df_albums


def process_artists(df):
    df_items_exploded = df.select(explode(col("items")).alias("item"))    
    df_artists_exploded = df_items_exploded.select(explode(col("item.track.artists")).alias("artist"))
    
    df_artists = df_artists_exploded.select(
        col("artist.id").alias("artist_id"),
        col("artist.name").alias("artist_name"),
        col("artist.external_urls.spotify").alias("external_url")
    ).drop_duplicates(["artist_id"])
    
    return df_artists

def process_songs(df):
    df_exploded = df.select(explode(col("items")).alias("item"))
    
    df_songs = df_exploded.select(
        col("item.track.id").alias("song_id"),
        col("item.track.name").alias("song_name"),
        col("item.track.duration_ms").alias("duration_ms"),
        col("item.track.external_urls.spotify").alias("url"),
        col("item.track.popularity").alias("popularity"),
        col("item.added_at").alias("song_added"),
        col("item.track.album.id").alias("album_id"),
        col("item.track.artists")[0]["id"].alias("artist_id")
    ).drop_duplicates(["song_id"])
    
    df_songs = df_songs.withColumn("song_added", to_date(col("song_added")))
    
    return df_songs


In [25]:
# Process Data

albums_df = process_albums(spotify_df)
albums_df.show()

+--------------------+--------------------+------------+------------+--------------------+
|            album_id|          album_name|release_date|total_tracks|                 url|
+--------------------+--------------------+------------+------------+--------------------+
|03guxdOi12XJbnvxv...|           Submarine|  2024-05-31|          14|https://open.spot...|
|07V9HO6Djetw5j5lX...|    So Close To What|  2025-02-20|          16|https://open.spot...|
|0QLILSOq1IqAqx62d...|     back to friends|  2024-12-27|           1|https://open.spot...|
|0Rfqw98N7kZ4qVPGW...|      Strange Trails|  2015-04-07|          14|https://open.spot...|
|0XKconI47eiBP6qPY...|The Rise and Fall...|  2023-09-21|          14|https://open.spot...|
|0hBRqPYPXhr1RkTDG...|The Secret of Us ...|  2024-10-18|          20|https://open.spot...|
|0hvT3yIEysuuvkK73...|                 GNX|  2024-11-22|          12|https://open.spot...|
|0sWJ3v0FROzcNzyjN...|           LA CIUDAD|  2024-12-24|          12|https://open.spot...|

In [18]:
artists_df = process_artists(spotify_df)
artists_df.show()

+--------------------+-----------------+--------------------+
|           artist_id|      artist_name|        external_url|
+--------------------+-----------------+--------------------+
|0du5cEVh5yTK9QJze...|       Bruno Mars|https://open.spot...|
|0fTSzq9jAh4c36UVb...|      Alex Warren|https://open.spot...|
|12GqGscKJx3aE4t07...|       Peso Pluma|https://open.spot...|
|1HY2Jd0NmPuamShAr...|        Lady Gaga|https://open.spot...|
|1Xyo4u8uXC1ZmMpat...|       The Weeknd|https://open.spot...|
|1Yj5Xey7kTwvZla8s...|          Cris Mj|https://open.spot...|
|1iCnM8foFssWlPRLf...|       Gigi Perez|https://open.spot...|
|1jiZvw42D4oquLl24...|    Lefty Gunplay|https://open.spot...|
|1oSPZhvZMIrWW5I41...|            Jimin|https://open.spot...|
|1pBLC0qVRTB5zVMut...|   Lily-Rose Depp|https://open.spot...|
|22wbnEMDvgVIAGdFe...|     Benson Boone|https://open.spot...|
|250b0Wlc5Vk0CoUsa...|           JENNIE|https://open.spot...|
|2OFij1ppCwYFCOTq7...|     bees & honey|https://open.spot...|
|2YZyLoL

In [27]:
songs_df = process_songs(spotify_df)
songs_df.show()

+--------------------+--------------------+-----------+--------------------+----------+----------+--------------------+--------------------+
|             song_id|           song_name|duration_ms|                 url|popularity|song_added|            album_id|           artist_id|
+--------------------+--------------------+-----------+--------------------+----------+----------+--------------------+--------------------+
|0DMJz677XfXiiruhI...|            VITAMINA|     208743|https://open.spot...|        92|2025-04-28|6jf5qinEHzsxyBQgv...|3Y9A8EQQtWU8RStiT...|
|0FIDCNYYjNvPVimz5...|Timeless (feat Pl...|     256000|https://open.spot...|        95|2025-04-28|3OxfaVgvTxUTy7276...|1Xyo4u8uXC1ZmMpat...|
|0FTmksd2dxiE5e3rW...|     back to friends|     199032|https://open.spot...|        93|2025-04-28|0QLILSOq1IqAqx62d...|4G9NDjRyZFDlJKMRL...|
|0WbMK4wrZ1wFSty9F...|    Good Luck, Babe!|     218423|https://open.spot...|        95|2025-04-28|1WAjjRMfZjEXtB0lQ...|7GlBOeep6PqTfFi59...|
|0aB0v4027ukV

### Write Transformed Data to S3

In [32]:
def write_to_s3(df, path_suffix, format_type="csv"):
    
    # Convert back to DynamicFrame
    dynamic_frame = DynamicFrame.fromDF(df, glueContext, "dynamic_frame")
    glueContext.write_dynamic_frame.from_options(
        frame=dynamic_frame,
        connection_type="s3",
        connection_options={"path": f"s3://spotify-bucket-jay/transformed_data/{path_suffix}/"},
        format=format_type
    )

In [34]:
write_to_s3(albums_df, "albums/album_transformed_{}".format(datetime.now().strftime("%Y-%m-%d")), "csv")
write_to_s3(artists_df, "artists/artist_transformed_{}".format(datetime.now().strftime("%Y-%m-%d")), "csv")
write_to_s3(songs_df, "songs/songs_transformed_{}".format(datetime.now().strftime("%Y-%m-%d")), "csv")

In [35]:
job.commit()